# Configuration

In [ ]:
API_KEY = "API 키 입력하세요"

# 1.Test Call

In [ ]:
!pip install openai # 라이브러리 설치

In [ ]:
from openai import OpenAI

# OpenAI 클라이언트 생성 (테스트용이니 키를 직접 넣는 버전)
client = OpenAI(api_key=API_KEY)  # 여기에 openai API키 넣기
prompt = f"""
너의 소개를 복잡하게 해줘!! 반말하지 말고 존댓말 써
"""

# OpenAI API 호출 (Responses API + JSON 형식 강제)
response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt,
)

response.output_text

'안녕하세요! 저는 OpenAI에서 개발한 최첨단 인공지능 언어 모델인 ChatGPT입니다. 자연어 처리를 기반으로 하여 다양한 주제에 대해 심도 있는 대화를 나누고, 복잡한 질문에 대해서도 논리적이고 체계적인 답변을 제공할 수 있습니다. 최신 언어 이해 기술과 방대한 데이터 학습을 통해, 여러분의 다양한 요구에 맞추어 맞춤형 정보 전달, 창의적인 아이디어 생성, 문제 해결 지원 등 다방면에서 도움을 드릴 수 있습니다. 언제든지 궁금한 점이 있거나 도움이 필요하시면 편하게 말씀해 주시기 바랍니다.'

# 2.(예제1) 데이터 분류: 감성분석 (단일)

## Load Libraries & Data

In [ ]:
# =============================
# 0) 라이브러리 설치 (노트북 첫 셀에서 한 번만)
# =============================
!pip install datasets pandas openai

In [ ]:
from datasets import load_dataset # huggingface library. 각종 공개 데이터를 쉽게 쓸 수 있게 해줌.
import pandas as pd

# =============================
# 1) SST-2 데이터셋 로드
# =============================
sst2 = load_dataset("stanfordnlp/sst2")

# train / validation / test 중 train만 사용
train_ds = sst2["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

## EDA/Preprocessing

EDA를 하는 이유?

In [ ]:
# =============================
# 2) pandas DataFrame으로 변환
# =============================
train_df = train_ds.to_pandas() #dataset 객체>>pandas로

print("=== 상위 5행 ===")
print(train_df)

=== 상위 5행 ===
         idx                                           sentence  label
0          0       hide new secretions from the parental units       0
1          1               contains no wit , only labored gags       0
2          2  that loves its characters and communicates som...      1
3          3  remains utterly satisfied to remain the same t...      0
4          4  on the worst revenge-of-the-nerds clichés the ...      0
...      ...                                                ...    ...
67344  67344                               a delightful comedy       1
67345  67345                   anguish , anger and frustration       0
67346  67346  at achieving the modest , crowd-pleasing goals...      1
67347  67347                                  a patient viewer       1
67348  67348  this new jangle of noise , mayhem and stupidit...      0

[67349 rows x 3 columns]


In [ ]:
print("컬럼:", train_df.columns)

컬럼: Index(['idx', 'sentence', 'label'], dtype='object')


In [ ]:
train_df.value_counts()

,,,count
idx,sentence,label,
67348,"this new jangle of noise , mayhem and stupidity must be a serious contender for the title .",0,1
0,hide new secretions from the parental units,0,1
1,"contains no wit , only labored gags",0,1
2,that loves its characters and communicates something rather beautiful about human nature,1,1
3,remains utterly satisfied to remain the same throughout,0,1
...,...,...,...
9,are more deeply thought through than in most ` right-thinking ' films,1,1
8,a depressed fifteen-year-old 's suicidal poetry,0,1
7,of saucy,1,1


In [ ]:
# =============================
# 3) 레이블 분포/비율 보기
# =============================
print("=== 레이블 분포 (개수) ===")
print(train_df["label"].value_counts())

=== 레이블 분포 (개수) ===
label
1    37569
0    29780
Name: count, dtype: int64


In [ ]:
print("=== 레이블 분포 (비율) ===")
print(train_df["label"].value_counts(normalize=True)) # 데이터 균형/불균형 확인

=== 레이블 분포 (비율) ===
label
1    0.557826
0    0.442174
Name: proportion, dtype: float64


In [ ]:
# 레이블 의미 매핑 (SST-2)
label_map = {
    0: "negative",
    1: "positive"
}

# =============================
# 4) 텍스트 길이(문자 수) 분포 보기
# =============================

train_df["text_len"] = train_df['sentence'].str.len()

print("=== 텍스트 길이 기초 통계 ===")
print(train_df["text_len"].describe())
print()

=== 텍스트 길이 기초 통계 ===
count    67349.00000
mean        53.50559
std         43.40738
min          2.00000
25%         21.00000
50%         39.00000
75%         74.00000
max        268.00000
Name: text_len, dtype: float64



In [ ]:
train_df["text_len"] = train_df['sentence'].str.len()

In [ ]:
train_df["text_len"].describe()

,text_len
count,67349.00000
mean,53.50559
std,43.40738
min,2.00000
25%,21.00000
50%,39.00000
75%,74.00000
max,268.00000


## Classification

In [ ]:
# =============================
# 5) 1번째 행을 LLM으로 감성 분류하기 (OpenAI gpt-4.1-mini)
# =============================
from openai import OpenAI
import json

# 1) OpenAI 클라이언트 생성
client = OpenAI(api_key=API_KEY)

# 2) 1번째 문장 텍스트와 정답 레이블 확인
row = train_df.iloc[1]
text = row['sentence']

true_label_name = label_map[row["label"]]

print("=== 0번째 문장 ===")
print(text)

print("정답 레이블 (이름):", true_label_name)


=== 0번째 문장 ===
contains no wit , only labored gags 
정답 레이블 (이름): negative


In [ ]:
# 3) LLM에게 감성분석 요청 프롬프트
prompt = f"""
당신은 감정 분류자입니다.

다음 문장을 다음 라벨 중 하나로 분류하세요:
- 부정
- 긍정
문장:
"{text}"
"""

# 4) OpenAI API 호출 (Responses API + JSON 형식 강제)
response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt,
    temperature = 0, #분류를 위한 안정적인 답변
    top_p = 1 # 단어풀에서 괜찮다 싶은 단어들을 최대한 사용하여 답변

)

# 5) 응답에서 JSON 파싱해서 label만 뽑기
response.output_text

'라벨: 부정'

In [ ]:
#Q. f"" 이게뭔가요??
#A.“중괄호 {} 안에 변수를 끼워 넣는 문자열 템플릿”
name = "성민"
age = 20
print(f"제 이름은 {name}이고, 나이는 {age}살입니다.")

# 3.(Lab1) 데이터 분류: 토픽분석 (단일)

- 당신은 Text 데이터 처리 전문 데이터 사이언티스트입니다.
- PR 부서에서 뉴스데이터들을 주며 아래와 같은 문의를 남겼습니다.



- Q) 0번쨰 멘션의 토픽은 어떤 종류의 토픽인지?

  데이터의 0번째 기사가 아래 토픽 중 어떤 토픽에 해당하는 기사인지 분류를 요청했습니다.

  토픽: World, Sports, Business, Sci/Tech

## Load Libraries & Data

In [ ]:
from datasets import load_dataset
import pandas as pd

# 1) 뉴스 데이터셋 로드 (ag_news)
dataset = load_dataset("ag_news")

# train / test 둘 중 train만 일단 사용
train_ds = dataset["train"]

# 2) pandas DataFrame으로 변환
train_df = train_ds.to_pandas()


train_df

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [ ]:

#==============================lab==============================#
print("=== 상위 5행 ===")
print(train_df.head()) # 상위 5개 행을 확인하는 코드?

=== 상위 5행 ===
                                                text  label
0  Wall St. Bears Claw Back Into the Black (Reute...      2
1  Carlyle Looks Toward Commercial Aerospace (Reu...      2
2  Oil and Economy Cloud Stocks' Outlook (Reuters...      2
3  Iraq Halts Oil Exports from Main Southern Pipe...      2
4  Oil prices soar to all-time record, posing new...      2


## EDA/Preprocessing

In [ ]:
# 3) 레이블 분포 보기
#==============================lab==============================#
train_df["label"].value_counts() # 값별 분포 확인?

,count
label,
2,30000
3,30000
1,30000
0,30000


In [ ]:
#==============================lab==============================#
# 4) 레이블별 비율
print(train_df["label"].value_counts(normalize=True))
# 다른 방법
# total = len(train_df["label"])
# train_df["label"].value_counts()/total

label
2    0.25
3    0.25
1    0.25
0    0.25
Name: proportion, dtype: float64


In [ ]:
# 5) 텍스트 길이(문자 수) 분포 보기
text_col = "text"

#==============================lab==============================#
# text 길이를 새로운 컬럼으로 생성하기.
train_df["text_len"] = train_df[text_col].str.len()   #hint: len 사용

train_df["text_len"]

,text_len
0,144
1,266
2,232
3,256
4,234
...,...
119995,221
119996,308
119997,247
119998,370


In [ ]:
#==============================lab==============================#
print("=== 텍스트 길이 기초 통계 ===")
print(train_df["text_len"].describe())

=== 텍스트 길이 기초 통계 ===
count    120000.000000
mean        236.477525
std          66.509741
min         100.000000
25%         196.000000
50%         232.000000
75%         266.000000
max        1012.000000
Name: text_len, dtype: float64


## Classification

In [ ]:
# =============================
# 6) 0번째 행을 LLM으로 토픽 분류하기 (OpenAI gpt-4.1-mini)
# =============================
from openai import OpenAI
import json


# 0번째 뉴스 텍스트와 정답 레이블 확인
row = train_df.iloc[0]
news_text = row[text_col]


# AG News 기준 레이블 매핑 (0~3)
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}
true_label_name = label_map[row["label"]]

# OpenAI 클라이언트 생성 (테스트용이니 키를 직접 넣는 버전)
client = OpenAI(api_key=API_KEY)
true_label_name = label_map[row["label"]]






print("=== 0번째 뉴스 원문 ===")
print(news_text)
print("정답 레이블 (이름):", true_label_name)


=== 0번째 뉴스 원문 ===
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
정답 레이블 (이름): Business


In [ ]:
# LLM에게 토픽 분류 요청 프롬프트

#==============================lab==============================#
prompt = f"""
너는 뉴스를 읽고 토픽을 분석해줘야해.

해당 뉴스의 토픽을 간결한 단어로 알려줘 토픽이란 단어는 쓸 필요 없어.

{news_text}
"""


"\n너는 뉴스를 읽고 토픽을 분석해줘야해.\n\n해당 뉴스의 토픽을 간결한 단어로 알려줘 토픽이란 단어는 쓸 필요 없어.\n\nWall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.\n"

In [ ]:

#==============================lab==============================#
# OpenAI API 호출


response = client.responses.create(
    model="gpt-4.1-mini", #gpt-4-mini 모델로 활용
    input=f"""
          너는 뉴스를 읽고 토픽을 분석해줘야해.

          해당 뉴스의 토픽을 간결한 단어를 선별해서 알려줘 토픽이란 단어는 쓸 필요 없어.

          {news_text}
          """,
    temperature = 0, #분류를 위한 안정적인 답변
    top_p = 1 # 단어풀에서 괜찮다 싶은 단어들을 최대한 사용하여 답변
)

# 응답에서 JSON 파싱해서 label만 뽑기
pred_text = response.output[0].content[0].text

print("=== LLM 예측 결과 ===")
print("예측 토픽:", pred_text)

=== LLM 예측 결과 ===
예측 토픽: 월가, 공매도, 투자자, 주식시장, 수익 회복


# 4.(예제2) 데이터 분류: 감성분석 (대용량)


## Load Libraries/EDA/Preprocessing

In [ ]:
!pip install datasets pandas openai

from datasets import load_dataset
import pandas as pd
from openai import OpenAI
import json
from tqdm import tqdm

client = OpenAI(api_key=API_KEY)

In [ ]:
# =============================
# 1) SST-2 데이터셋 로드
# =============================
sst2 = load_dataset("stanfordnlp/sst2")

# train / validation / test 중 train만 사용
train_ds = sst2["train"]

# =============================
# 2) pandas DataFrame으로 변환/ 100개 행 제한
# =============================
train_df = train_ds.to_pandas()
df_batch = train_df.iloc[:100].copy()

# =============================
# 3) 레이블 분포/비율 보기
# =============================
print("=== 레이블 분포 (개수) ===")
print(df_batch["label"].value_counts())

print("=== 레이블 분포 (비율) ===")
print(df_batch["label"].value_counts(normalize=True))

# 레이블 의미 매핑
label_map = {
    0: "negative",
    1: "positive"
}


=== 레이블 분포 (개수) ===
label
0    51
1    49
Name: count, dtype: int64
=== 레이블 분포 (비율) ===
label
0    0.51
1    0.49
Name: proportion, dtype: float64


## Classification

In [ ]:
import json
from openai import OpenAI

client = OpenAI(api_key=API_KEY)

# 1) 배치df 에서 문장 리스트 뽑기
sentences = df_batch["sentence"].tolist()

# 2) system 프롬프트
system_msg = """
[ROLE]
You are a sentiment classifier.
[/ROLE]

[INSTRUCTIONS]
- Classify the sentiment of English sentences.
- Valid labels are only "negative" or "positive".
- When the user sends a JSON array of sentences,
  you must respond with a JSON object.
- In the JSON object, each key must be the exact input sentence string,
  and each value must be its sentiment label ("negative" or "positive").
- Do NOT add any explanations or extra text outside of the JSON.
[/INSTRUCTIONS]

[OUTPUT FORMAT]
{
  "sentence_1": "negative",
  "sentence_2": "positive",
  ...
}
[/OUTPUT FORMAT]
"""

# 3) few-shot 예시
fewshot_input = [
    "This movie was a complete waste of time.",
    "I absolutely loved this film, it was fantastic!",
    "The plot was boring and the acting was terrible."
]

fewshot_output = {
    "This movie was a complete waste of time.": "negative",
    "I absolutely loved this film, it was fantastic!": "positive",
    "The plot was boring and the acting was terrible.": "negative"
}

messages = [
    # 역할/규칙 정의
    {"role": "system", "content": system_msg},

    # ==== Few-shot 예시: 입력(JSON 배열) ====
    {
        "role": "user",
        "content": json.dumps(fewshot_input)
    },
    # ==== Few-shot 예시: 출력(JSON object) ====
    {
        "role": "assistant",
        "content": json.dumps(fewshot_output)
    },

    # ==== 실제 우리가 분류하고 싶은 문장들 (JSON 배열) ====
    {
        "role": "user",
        "content": json.dumps(sentences)
    },
]

In [ ]:
# 4) API 한 번 호출해서, 문장→라벨 딕셔너리(JSON object) 받기
response = client.responses.create(
    model="gpt-4.1-mini",
    input=messages,
    temperature=0, #분류를 위한 안정적인 답변
    top_p=1,
)

raw_text = response.output_text
sentence_to_label = json.loads(raw_text)

# 5) df_batch에 예측 라벨 붙이기
#    key가 문장(text)이기 때문에, sentence 컬럼을 키로 써서 매핑
df_batch["pred_label_name"] = df_batch["sentence"].map(sentence_to_label)

# 정답 라벨 문자열
df_batch["true_label_name"] = df_batch["label"].map(label_map)

# 6) 정확도 계산
accuracy = (df_batch["pred_label_name"] == df_batch["true_label_name"]).mean()
print(f"Accuracy: {accuracy:.4f}")

# 7) 샘플 출력
print(df_batch[["sentence", "label", "true_label_name", "pred_label_name"]].head(10))


Accuracy: 0.9900
                                            sentence  label true_label_name  \
0       hide new secretions from the parental units       0        negative   
1               contains no wit , only labored gags       0        negative   
2  that loves its characters and communicates som...      1        positive   
3  remains utterly satisfied to remain the same t...      0        negative   
4  on the worst revenge-of-the-nerds clichés the ...      0        negative   
5  that 's far too tragic to merit such superfici...      0        negative   
6  demonstrates that the director of such hollywo...      1        positive   
7                                          of saucy       1        positive   
8   a depressed fifteen-year-old 's suicidal poetry       0        negative   
9  are more deeply thought through than in most `...      1        positive   

  pred_label_name  
0        negative  
1        negative  
2        positive  
3        negative  
4        nega

In [ ]:
# Q map 함수 이해가 잘 안가요..
# 예제를 봅시다!

import pandas as pd

df = pd.DataFrame({
    "fruit": ["apple", "banana", "apple", "orange"]
})

fruit_color = {
    "apple": "red",
    "banana": "yellow",
    "orange": "orange"
}

df["color"] = df["fruit"].map(fruit_color)
print(df)

    fruit   color
0   apple     red
1  banana  yellow
2   apple     red
3  orange  orange


# 5.(LAB 2) 데이터 분류: 감성분석 (대용량)

## Load Libraries/EDA/Preprocessing

In [ ]:
from datasets import load_dataset
import pandas as pd

# =============================
# 1) 뉴스 토픽 데이터셋 로드 (AG News)
# =============================
ag_news = load_dataset("ag_news")

# train / test 중 train만 사용
train_ds = ag_news["train"]

# =============================
# 2) pandas DataFrame으로 변환 / 100개 행 제한
# =============================
train_df = train_ds.to_pandas()
df_batch = train_df.iloc[:100].copy()

# =============================
# 3) 레이블 분포 / 비율 보기
# =============================
print("=== 레이블 분포 (개수) ===")
print(df_batch["label"].value_counts())

print("=== 레이블 분포 (비율) ===")
print(df_batch["label"].value_counts(normalize=True))

# 레이블 의미 매핑 (AG News 기준)
# 0: World, 1: Sports, 2: Business, 3: Sci/Tech
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}


=== 레이블 분포 (개수) ===
label
2    78
3    22
Name: count, dtype: int64
=== 레이블 분포 (비율) ===
label
2    0.78
3    0.22
Name: proportion, dtype: float64


In [ ]:
df_batch

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
95,Gene Blocker Turns Monkeys Into Workaholics - ...,3
96,Dolphins Too Have Born Socialites (Reuters) Re...,3
97,"What's in a Name? Well, Matt Is Sexier Than Pa...",3
98,UK Scientists Allowed to Clone Human Embryos (...,3


## Classification

In [ ]:
import json
from openai import OpenAI

client = OpenAI(api_key=API_KEY)

# 1) 배치df 에서 문장 리스트 뽑기
sentences = df_batch["text"].tolist()

#==============================lab==============================#
# 2) system 프롬프트 (뉴스 토픽 분류기로 역할 지정)
system_msg = """
[ROLE]
You are a news topic classifier.
[/ROLE]

[INSTRUCTIONS]
- Categorize topics into just four defined news topics.
- Valid labels are only "World" or "Sports" or "Business" or "Sci/Tech".
- When the user sends a JSON array of sentences,
  you must respond with a JSON object.
- In the JSON object, each key must be the exact input sentence string,
  and each value must be its sentiment label ("World" or "Sports" or "Business" or "Sci/Tech").
- Do NOT add any explanations or extra text outside of the JSON.

  0: "World",
  1: "Sports",
  2: "Business",
  3: "Sci/Tech"

[/INSTRUCTIONS]

[OUTPUT FORMAT]
{
  0: "World",
  1: "Sports",
  2: "Business",
  3: "Sci/Tech"
  ...
}
[/OUTPUT FORMAT]
"""


In [ ]:
#==============================lab==============================#
# 3) few-shot 예시
fewshot_input = [
  "It's amazing what's happening on Earth these days.",
  "A new record was set at the Olympics.",
  "I think my health has deteriorated. This sport seems to be all the rage these days."
  "I see companies are getting investments this way these days.",
  "I heard there's a new inventor. Science is truly amazing."
]

fewshot_output = {
  "It's amazing what's happening on Earth these days.":"World",
  "A new record was set at the Olympics.":"Sports",
  "I think my health has deteriorated. This sport seems to be all the rage these days.":"Sports",
  "I see companies are getting investments this way these days.":"Business",
  "I heard there's a new inventor. Science is truly amazing.":"Sci/Tech"
}

# 기존 감성분석 코드와 동일한 messages 구조 유지
messages = [
    # 시스템 메세지 작성
    {"role": "system", "content": system_msg},

    # ==== Few-shot 예시: 입력(JSON 배열) ====
    # user message 작성
    {
        "role": "user",
        "content": json.dumps(fewshot_input)
    },
    # ==== Few-shot 예시: 출력(JSON object) ====
    { # assistant message 작성
        "role": "assistant",
        "content": json.dumps(fewshot_output)
    },
    # ==== 실제 우리가 분류하고 싶은 문장들 (JSON 배열) ====
    {
        "role": "user",
        "content": json.dumps(sentences)
    },
]

In [ ]:
#==============================lab==============================#
# 4) API 한 번 호출해서, 뉴스텍스트→토픽 라벨 딕셔너리(JSON object) 받기
response = client.responses.create(
    model="gpt-4.1-mini",
    input=messages,
    temperature=0, #분류를 위한 안정적인 답변
    top_p=1,
)

raw_text = response.output_text.strip()

# JSON 파싱: { "문장": "토픽" } 딕셔너리
sentence_to_topic = json.loads(raw_text)

# 5) df_batch에 예측 라벨 붙이기
#    key가 문장(text)이기 때문에, text 컬럼을 키로 써서 매핑
df_batch["pred_topic_name"] = df_batch["text"].map(sentence_to_topic)

# 정답 라벨 문자열
df_batch["true_topic_name"] = df_batch["label"].map(label_map)

# 6) 정확도 계산
accuracy = (df_batch["pred_topic_name"] == df_batch["true_topic_name"]).mean()
print(f"Accuracy: {accuracy:.4f}")

# 7) 샘플 출력
print(df_batch[["text", "label", "true_topic_name", "pred_topic_name"]].head(10))


Accuracy: 0.6900
                                                text  label true_topic_name  \
0  Wall St. Bears Claw Back Into the Black (Reute...      2        Business   
1  Carlyle Looks Toward Commercial Aerospace (Reu...      2        Business   
2  Oil and Economy Cloud Stocks' Outlook (Reuters...      2        Business   
3  Iraq Halts Oil Exports from Main Southern Pipe...      2        Business   
4  Oil prices soar to all-time record, posing new...      2        Business   
5  Stocks End Up, But Near Year Lows (Reuters) Re...      2        Business   
6  Money Funds Fell in Latest Week (AP) AP - Asse...      2        Business   
7  Fed minutes show dissent over inflation (USATO...      2        Business   
8  Safety Net (Forbes.com) Forbes.com - After ear...      2        Business   
9  Wall St. Bears Claw Back Into the Black  NEW Y...      2        Business   

  pred_topic_name  
0        Business  
1        Business  
2        Business  
3           World  
4        Busi